## Importing libraries

In [23]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference
import string
import xlsxwriter

### Working with excel file

##### We have added excel file from local disk, however can be added to jupyter notebook using json, ODBC, redshift etc

In [24]:
# Importing excel file
wbs=pd.read_excel('sales_records.xlsx',sheet_name=None)

In [25]:
# checking number of sheets in the excel file and sheet
num_of_sheets=len(wbs.keys())
sheet_names=wbs.keys()
print('number of sheets: '+ str(num_of_sheets) )
print('sheet names: '+ str(sheet_names) )

number of sheets: 1
sheet names: dict_keys(['sales_records'])


In [26]:
# number of rows in each sheet
for sheet_name in wbs.keys():
    print(str(sheet_name) + " has " + str(len(wbs[sheet_name].index)) + " rows")

sales_records has 100000 rows


In [27]:
# checking head/tail on both sheets 
def display_first_n_rows(wbs,sheet_names,n):
    print(wbs[sheet_names].head(n))

In [28]:
# Checking first n rows of all the sheets
for i in range(num_of_sheets):
    print("Results of sheet:" + str(i) + '\n')
    print(display_first_n_rows(wbs,list(wbs.keys())[i],10))

Results of sheet:0

                              Region                Country      Item Type  \
0       Middle East and North Africa             Azerbaijan         Snacks   
1  Central America and the Caribbean                 Panama      Cosmetics   
2                 Sub-Saharan Africa  Sao Tome and Principe         Fruits   
3                 Sub-Saharan Africa  Sao Tome and Principe  Personal Care   
4  Central America and the Caribbean                 Belize      Household   
5                             Europe                Denmark        Clothes   
6                             Europe                Germany      Cosmetics   
7       Middle East and North Africa                 Turkey         Fruits   
8                             Europe         United Kingdom         Snacks   
9                               Asia             Kazakhstan      Cosmetics   

  Sales Channel Order Priority           Order Date   Order ID  \
0        Online              C  2014-08-10 00:00:00  53

In [29]:
# Concatenate both the sheets
df = pd.concat(wbs, ignore_index=True)

In [30]:
df.head(5),df.tail(5)

(                              Region                Country      Item Type  \
 0       Middle East and North Africa             Azerbaijan         Snacks   
 1  Central America and the Caribbean                 Panama      Cosmetics   
 2                 Sub-Saharan Africa  Sao Tome and Principe         Fruits   
 3                 Sub-Saharan Africa  Sao Tome and Principe  Personal Care   
 4  Central America and the Caribbean                 Belize      Household   
 
   Sales Channel Order Priority           Order Date   Order ID  \
 0        Online              C  2014-08-10 00:00:00  535113847   
 1       Offline              L            2/22/2015  874708545   
 2       Offline              M  2015-09-12 00:00:00  854349935   
 3        Online              M            9/17/2014  892836844   
 4       Offline              H  2010-04-02 00:00:00  129280602   
 
              Ship Date  Units Sold  Unit Price  Unit Cost  Total Revenue  \
 0           10/23/2014         934      15

## Making pivot table

In [31]:
# Check column names
df.columns

Index(['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
       'Order Date', 'Order ID', 'Ship Date', 'Units Sold', 'Unit Price',
       'Unit Cost', 'Total Revenue', 'Total Cost', 'Total Profit'],
      dtype='object')

In [32]:
#Total profit by region
report_table_1= df.pivot_table(index=['Region'],
                                      values=['Total Profit'],columns='Item Type',
                                      aggfunc='sum',margins=True,margins_name='Grand Total').round(0)

In [33]:
#Total revenue By country
report_table_2= df.pivot_table(index=['Sales Channel'],
                                      values=['Total Profit'],columns='Item Type',
                                      aggfunc='sum',margins=True,margins_name='Grand Total').round(0)

In [34]:
#Total profit by order priority
report_table_3=df.pivot_table(index=['Item Type',],
                                      values=['Total Profit'],columns='Order Priority',
                                      aggfunc='sum',margins=True,margins_name='Grand Total').round(0)

In [35]:
#Total Revenue by Sales Channel
report_table_4= df.pivot_table(index=['Region'],
                                      values=['Units Sold','Total Revenue','Total Cost','Total Profit','Unit Price'],
                                      aggfunc='mean').round(0)

In [36]:
# we can also add multiple indexes and values for more informative reports
report_table_5= df.pivot_table(index=['Region','Sales Channel'],
                                      values=['Units Sold','Total Revenue','Total Cost','Total Profit','Unit Price'],
                                      aggfunc='mean').round(0)

In [37]:
# create a excel writer object
with pd.ExcelWriter("Summary.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    report_table_1.to_excel(writer, sheet_name="profits_regions")
    report_table_2.to_excel(writer, sheet_name="Sales_profit")
    report_table_3.to_excel(writer, sheet_name="Item type profit")
    report_table_4.to_excel(writer, sheet_name="Regions")
    report_table_5.to_excel(writer, sheet_name="Regions_SalesChannel")

In [42]:
# Changing Font
wb = load_workbook('Summary.xlsx')
sheet = wb['profits_regions']
sheet['A1'] = 'Region wise profit'
sheet['A2'] = 'Regions Vs Profits'
sheet['A1'].font = Font('Arial', bold=True, size=20)
sheet['A2'].font = Font('Arial', bold=True, size=10)
wb.save('Summary.xlsx')